In [1]:
import pandas as pd

In [2]:
import pickle

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [5]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    df = df[(df.duration >=1) & (df.duration <= 60)]
    categorical = ['PULocationID','DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df

In [6]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [7]:
len(df_train), len(df_val)

(73908, 61921)

In [8]:
df_train['PU_DO'] = df_train['PULocationID'] + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + df_val['DOLocationID']

In [9]:
categorical = ['PU_DO']#['PULocationID','DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()
train_dict = df_train[categorical+numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dicts = df_val[categorical+numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [10]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [11]:
lr = LinearRegression()
lr.fit(X_train,y_train)
y_pred = lr.predict(X_val)
mean_squared_error(y_val, y_pred)

60.64285040548521

In [12]:
lr = Lasso(alpha=0.005)
lr.fit(X_train,y_train)
y_pred = lr.predict(X_val)
mean_squared_error(y_val, y_pred)

114.34587003691898

In [13]:
lr = Ridge(alpha=0.01)
lr.fit(X_train,y_train)
y_pred = lr.predict(X_val)
mean_squared_error(y_val, y_pred)

57.00202336636105

In [14]:
with open('./models/lin_reg.bin','wb') as f_out:
    pickle.dump((dv,lr), f_out)